# 1. Az adatbázis betöltése.

*   HuggingFace-en elérhető, CommonCrawl-ról letöltött cikkek listája (~700k).
*   TO DO: adatbetöltés mélytanulás beadandó alapján.





In [ ]:
import requests
import numpy as np

# Define the API endpoint URL
api_url = "https://datasets-server.huggingface.co/rows?dataset=cc_news&config=plain_text&split=train&offset=0&limit=100"

try:
    # Send a GET request to the API
    response = requests.get(api_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON data if the API returns JSON
        data = response.json()

        # Convert JSON data to a NumPy array
        numpy_array = np.array(data)

        # Now you can work with the data
        # Example: Print the data
        print(numpy_array)

    else:
        print(f"Request failed with status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")


{'features': [{'feature_idx': 0, 'name': 'title', 'type': {'dtype': 'string', '_type': 'Value'}}, {'feature_idx': 1, 'name': 'text', 'type': {'dtype': 'string', '_type': 'Value'}}, {'feature_idx': 2, 'name': 'domain', 'type': {'dtype': 'string', '_type': 'Value'}}, {'feature_idx': 3, 'name': 'date', 'type': {'dtype': 'string', '_type': 'Value'}}, {'feature_idx': 4, 'name': 'description', 'type': {'dtype': 'string', '_type': 'Value'}}, {'feature_idx': 5, 'name': 'url', 'type': {'dtype': 'string', '_type': 'Value'}}, {'feature_idx': 6, 'name': 'image_url', 'type': {'dtype': 'string', '_type': 'Value'}}], 'rows': [{'row_idx': 0, 'row': {'title': 'Daughter Duo is Dancing in The Same Company', 'text': 'There\'s a surprising twist to Regina Willoughby\'s last season with Columbia City Ballet: It\'s also her 18-year-old daughter Melina\'s first season with the company. Regina, 40, will retire from the stage in March, just as her daughter starts her own career as a trainee. But for this one se

# 2. GPT-2 teszt modell meghívása HuggingFace-en keresztül.

In [ ]:
# install transformers
!pip install transformers

In [11]:
from pprint import pprint
from transformers import pipeline
import torch
from transformers import AutoTokenizer

model=''

# create pieline for generating text
gen = pipeline('text-generation', model=model) # uses GPT-2

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


(…)ingface.co/gpt2/resolve/main/config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

(…)gpt2/resolve/main/generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

(…)gingface.co/gpt2/resolve/main/vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

(…)gingface.co/gpt2/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)face.co/gpt2/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [2]:
gen("I am using transformers text-generation pipeline from Hugging Face library to generate")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input length of input_ids is 26, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


[{'generated_text': "Please write a 100-word article about Apple Inc.'s 2020 Q3 earnings report.\n"}]

In [3]:
prompt = "I am using transformers text-generation pipeline from Hugging Face library to generate"
pprint(gen(prompt,num_return_sequences = 3, max_length = 30))

Input length of input_ids is 26, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


[{'generated_text': "Please write a 100-word article about Apple Inc.'s 2020 "
                    'Q3 earnings report.\n'}]


In [ ]:
# download data from provided link
!wget https://www.dropbox.com/s/0xylltp1vxf31vj/robert_frost.txt?dl=0

In [9]:
# import required libraries
import textwrap
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!cat /content/robert_frost.txt?dl=0

In [11]:
lines = [line.rstrip() for line in open("robert_frost.txt?dl=0")]
lines = [line for line in lines if len(line) > 0]

In [ ]:
pprint(gen(lines[0],num_return_sequences = 3, max_length = 30))

In [14]:
def wrap(x):
  return textwrap.fill(x, replace_whitespace = False, fix_sentence_endings = True)

In [ ]:
prompt  = "Current state of deep learning and its potential impact on various industries"
out = gen(prompt, max_length = 300)
print(wrap(out[0]['generated_text']))

# 3. Llama 2

## 3.1. Llama 2 használata szöveg generálásra -- Működik
  Forrás: https://towardsdatascience.com/challenges-in-stop-generation-within-llama-2-25f5fea8dea2

In [1]:
!pip install xformers

In [3]:
import transformers
from torch import cuda
from pprint import pprint

# bit and byte configuration
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype= 'float16'  # Computation type
)
model_id= "NousResearch/Llama-2-7b-chat-hf"
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# Llama 2 Tokenizer
tokenizer =transformers.AutoTokenizer.from_pretrained(model_id)

# Llama 2 Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.eval()

# Difine pipeline
gen = transformers.pipeline(model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    max_new_tokens=500,  # max number of tokens to generate in the output
    temperature=0.1,
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
pprint(gen("Can you write a 100-word article about Apple Inc.'s 2020 Q3 earnings report?"))

[{'generated_text': "Can you write a 100-word article about Apple Inc.'s 2020 "
                    'Q3 earnings report?\n'
                    ' nobody 100-word article:\n'
                    'Apple Inc. reported its Q3 2020 earnings, beating analyst '
                    'expectations with $53.3 billion in revenue and $11.7 '
                    'billion in profit. iPhone sales were up 1% '
                    'year-over-year, while services revenue hit a record $12.7 '
                    'billion. The company also announced a 4-for-1 stock '
                    'split, effective August 28. Despite the ongoing pandemic, '
                    "Apple's performance suggests the tech giant remains "
                    'resilient and adaptable in the face of global '
                    'challenges.'}]


In [10]:
pprint(gen("Can you write a 100-word article about Apple Inc.'s 2020 Q3 earnings report? Make sure that sensitive data (e.g. company name, company specific information) is masked."))

[{'generated_text': "Can you write a 100-word article about Apple Inc.'s 2020 "
                    'Q3 earnings report? Make sure that sensitive data (e.g. '
                    'company name, company specific information) is masked.\n'
                    '\n'
                    'Here is the article you requested:\n'
                    '\n'
                    'Apple Inc. released its 2020 Q3 earnings report on '
                    'Tuesday, revealing a significant increase in revenue and '
                    "profit. The company's net income for the quarter was "
                    '$12.7 billion, up from $8.8 billion in the same period '
                    'last year. Revenue also increased by 11% to $53.3 '
                    "billion. The company's iPhone sales were up 15% from last "
                    "year, while Mac sales increased by 25%. Apple's Services "
                    'segment also saw significant growth, with revenue up 14% '
                    "to $12.2 b

## 3.2. Llama 2 fine-tune-olása
*   Forrás: https://colab.research.google.com/drive104gE9FlBODgwBaPez8th6OEaneRL3heA#scrollTo=OJXpOgBFuSrc
*   Original forrás: https://colab.research.google.com/drive/1PEQyJO1-f6j0S_XJ8DV50NkpzasXkrzd?usp=sharing#scrollTo=OJXpOgBFuSrc


In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "cc_news/viewer/plain_text/train?row=0"

# Fine-tuned model name
new_model = "llama-2-7b-articles"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
#TBD